# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **1** 

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Afonso Ascensão|20240684|
|André Filipe Silva|20230972|
|Joana Esteves|20240746|
|Susana Reis|20240567|
|Tomás Figueiredo|20240941|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

In [ ]:
from neo4j import GraphDatabase
from pprint import pprint
from py2neo import Graph
import networkx as nx
import matplotlib.pyplot as plt
import time

In [ ]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [ ]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

# Functions to run queries in Neo4J

In [ ]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

In [ ]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

# 1. Explore the database

## 1.1 Explore schema:

In [ ]:
# Get "visualization" schema. This generates the visualization schema and is better ran using the browser, but we put the query here for completion.

query = """
    call db.schema.visualization;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Creating indexes for faster processing of queries
query1 = """CREATE INDEX beer_id_index IF NOT EXISTS FOR (b:BEERS) ON (b.id)"""
query2 = """CREATE INDEX brewery_id_index IF NOT EXISTS FOR (b:BREWERIES) ON (b.id)"""
query3 = """CREATE INDEX user_name_index IF NOT EXISTS FOR (u:USERS) ON (u.name)"""
query4 = """CREATE INDEX review_id_index IF NOT EXISTS FOR (r:REVIEWS) ON (r.id)"""
query5 = """CREATE INDEX city_name_index IF NOT EXISTS FOR (c:CITIES) ON (c.name)"""
query6 = """CREATE INDEX country_name_index IF NOT EXISTS FOR (c:COUNTRIES) ON (c.name)"""
query7 = """CREATE INDEX style_name_index IF NOT EXISTS FOR (s:STYLE) ON (s.name)"""

# Execute each query separately
execute_write(driver, query1)
execute_write(driver, query2)
execute_write(driver, query3)
execute_write(driver, query4)
execute_write(driver, query5)
execute_write(driver, query6)
execute_write(driver, query7)

In [ ]:
# Get number of observations 

query = """
        MATCH () RETURN count(*)
    """

result = execute_read(driver, query)

print(f"We have {result[0][0]} observations in the database.")

In [ ]:
# Check indexes

query = """
        call db.indexes;
    """

result = execute_read(driver, query)

pprint(result)

Indexes:
- Style, user, countries and cities id is the property name.
- Reviews, Breweries, Beers id id the property id.
- We have to make sure these properties have unique values.

In [ ]:
# Check constraints

query = """
        call db.constraints;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check all labels

query = """
        call db.labels;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Retrieve all nodes in the database
# group them by label and count how many nodes exist for each label

query = """
        MATCH (n)
        RETURN labels(n) AS node_label, count(n) AS node_count
        ORDER BY node_label ASC
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check all types of relationships

query = """
        call db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

## Properties for each Node

In [ ]:
# Check all properties of the BEERS node
query = """
MATCH (n:BEERS) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the BREWERIES node
query = """
MATCH (n:BREWERIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the USER node
query = """
MATCH (n:USER) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the REVIEWS node
query = """
MATCH (n:REVIEWS) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the CITIES node
query = """
MATCH (n:CITIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the COUNTRIES node
query = """
MATCH (n:COUNTRIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the STYLE node
query = """
MATCH (n:STYLE) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

### Finding all One-Directional Relationships

In [ ]:
query = """
MATCH (a)-[r]->(b)
WHERE NOT EXISTS {
    MATCH (b)-[r_reverse]->(a)
}
RETURN DISTINCT
  type(r) AS relationship_type,
  labels(a) AS start_node_labels,
  labels(b) AS end_node_labels
"""

result = execute_read(driver, query)

pprint(result)

### Finding all Bidirectional Relationships

In [ ]:
query = """
MATCH (a)-[r]->(b)
MATCH (b)-[r_reverse]->(a)
WHERE type(r) = type(r_reverse)
RETURN DISTINCT
  type(r) AS relationship_type,
  labels(a) AS start_node_labels,
  labels(b) AS end_node_labels
"""

result = execute_read(driver, query)
pprint(result)

### Finding Self-Referencing Relationships

From our schema visualization, CITIES seems to be referencing back to iself. Let's try to understand that.

In [ ]:
query = """
MATCH (c:CITIES)-[r]->(c:CITIES)
RETURN c, r
"""

result = execute_read(driver, query)
pprint(result)

From this query it seems like the CITIES [IN] CITIES relationship seen in the visualization is merely a graphical artifact.

From these queries we can conclude there are only one-directional relationships. We believe this makes sense, from our understanding of the database.

In [ ]:
# Check properties for each relationship

query = """
        MATCH ()-[r]->()
        RETURN DISTINCT type(r) AS relationship_type, keys(r) AS relationship_properties
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for isolated nodes

query = """
        MATCH (n)
        WHERE NOT EXISTS ((n)--())
        RETURN n
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Get what type of isolated nodes we have 

query = """
        MATCH (n)
        WHERE NOT EXISTS ((n)--())
        RETURN DISTINCT labels(n)
    """

result = execute_read(driver, query)

pprint(result)

- There are isolated nodes of all labels.

## 1.2 Check for inconsistencies: 

In [ ]:
# Checking for "corrupt" nodes - nodes with no labels

query = """
MATCH (n)
WHERE size(labels(n)) = 0
RETURN id(n) AS node_id, labels(n) AS labels, properties(n) AS properties
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Checking for nodes with no properties

query = """
MATCH (n)
WHERE size(keys(n)) = 0
RETURN id(n) AS node_id, labels(n) AS labels, properties(n) AS properties
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label COUNTRIES and same name

query = """
MATCH (c:COUNTRIES)
WITH c.name AS name, COUNT(c) AS count
WHERE count > 1
RETURN name, count;
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label CITIES and same name

query = """
    MATCH (ci:CITIES)
    WITH ci.name AS cityName, count(ci) AS cityCount
    WHERE cityCount > 1
    RETURN cityName, cityCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if one node is connected to more than one city

query = """
MATCH (c:CITIES)-[:IN]->(co:COUNTRIES)
WITH co.name AS name, COUNT(DISTINCT co) AS country_count, COLLECT(DISTINCT co) AS country_nodes
WHERE country_count > 1
RETURN name, country_count, country_nodes;
"""

result = execute_read(driver, query)

pprint(result)

Since each country is connected to one node, we can safely remove the duplicated CITIES and COUNTRIES nodes.

In [ ]:
# Check total nodes w/ label COUNTRIES 

query = """
MATCH (c:COUNTRIES)
RETURN COUNT(c) AS total_nodes;
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check total nodes w/ label CITIES

query = """
MATCH (c:CITIES)
RETURN COUNT(c) AS total_nodes;
"""

result = execute_read(driver, query)

pprint(result)

- Since that the only property for COUNTRIES, CITIES, STYLE and USER is name, having more than one node with the same name represents a duplicated node.
- There are several duplicated nodes for both COUNTRIES and CITIES labels.

In [ ]:
# Check for duplicated nodes w/ label STYLE and same name

query = """
    MATCH (st:STYLE)
    WITH st.name AS styleName,
    count(st) AS styleCount
    WHERE styleCount > 1
    RETURN styleName, styleCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label USER and same name

query = """
    MATCH (us:USER)
    WITH us.name AS userName,
    count(us) AS userCount
    WHERE userCount > 1
    RETURN userName, userCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY 

query = """
    MATCH (br:BREWERY)
    WITH br.id AS breweryId, 
        br.state AS breweryState,
        br.notes AS breweryNotes,
        br.types AS breweryTypes,
        br.name AS breweryName,
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryId, breweryName, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY and same id

query = """
    MATCH (br:BREWERY)
    WITH br.id AS breweryId, 
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryId, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY and same name

query = """
    MATCH (br:BREWERY)
    WITH br.name AS breweryName,
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryName, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BEERS 

query = """
    MATCH (be:BEERS)
    WITH be.id AS beerId, 
        be.retired AS beerRetired, 
        be.availability AS beerAvailability, 
        be.brewery_id AS beerBreId, 
        be.state AS beerState, 
        be.name AS beerName, 
        be.abv AS beerAbv, 
        be.notes AS beerNotes, 
    count(be) AS beerCount
    WHERE beerCount > 1
    RETURN beerId, beerName, beerCount
"""

result = execute_read(driver, query)

pprint(result)

- There are several nodes with the label BEERS duplicated for all properties.

In [ ]:
# Check for duplicated nodes w/ label REVIEWS 

query = """
    MATCH (re:REVIEWS)
    WITH re.text AS reviewTest,
        re.beer_id AS reviewBeerId,
        re.smell AS reviewSmell,
        re.look AS reviewLook,
        re.score AS reviewScore,
        re.taste AS reviewTaste,
        re.feel AS reviewFeel,
        re.overall AS reviewOverall,
        re.id AS reviewId,
        re.date AS reviewDate,
    count(re) AS reviewCount
    WHERE reviewCount > 1
    RETURN reviewId, reviewCount
"""

result = execute_read(driver, query)

pprint(result)

- There are several nodes with the label REVIEWS duplicated for all properties.

Considerations:
- One city should only be associated w/ one country.
- One brewery should only be associated w/ one city.
- One review should only be associated w/ one user.
- One review should only be associated w/ one beer.
- One beer should only be associated w/ one style
- One beer should only be associated w/ one brewery

In [ ]:
# Check if there is any brewery associated with more than one city
# Retrieve all brewery to city relationships
# In these relationships, count nº of cities for each brewery 

query = """
    MATCH (br:BREWERIES)-[:IN]->(ci:CITIES)
    WITH br, count(ci) AS city_count
    WHERE city_count > 1
    RETURN br.name, city_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there is any city associated with more than one country
# Retrieve all city to country relationships
# In these relationships, count nº of countries for each city 

query = """
    MATCH (ci:CITY)-[:IN]->(co:COUNTRIES)
    WITH ci, count(co) AS country_count
    WHERE country_count > 1
    RETURN ci.name, country_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there are REVIEWS posted by more than one USER
query = """
    MATCH (us:USER)-[:POSTED]->(re:REVIEW)
    WITH re, count(us) AS user_count
    WHERE user_count > 1
    RETURN re.id, user_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if each REVIEW is only linked ot one BEER
query = """
    MATCH (re:REVIEW)-[:REVIEWED]->(be:BEERS)
    WITH re, count(be) AS beer_count
    WHERE beer_count > 1
    RETURN re.id, beer_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if beers have more than one style
query = """
    MATCH (re:BEERS)-[:HAS_STYLE]->(st:STYLES)
    WITH re, count(st) AS style_count
    WHERE style_count > 1
    RETURN re.name, style_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if any beers are brewed in more than one brewery
query = """
    MATCH (be:BEERS)-[:BREWED]->(br:BREWERIES)
    WITH be, COUNT(br) AS brewery_count
    WHERE brewery_count > 1
    RETURN be.name, brewery_count
"""

result = execute_read(driver, query)

pprint(result)

Considerations:
- All breweries should be associated with a city.
- All cities should be associated with a country.
- All reviews should be associated to a user.
- All reviews shoul be associated with a beer. 
- All beers should be associated with a brewery. 



In [ ]:
# Check if there is any is any node w/ BREWERIES label
# that does not have the specified relationship to a node w/ label CITIES

query = """
    MATCH (br:BREWERIES)
    WHERE NOT EXISTS ((br)-[:IN]->(:CITIES))
    RETURN br
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there are any CITIES nodes not linked to COUNTRIES
query = """
    MATCH (ci:CITIES)
    WHERE NOT EXISTS ((ci)-[:IN]->(:COUNTRIES))
    RETURN ci
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there are reviews not linked to user
query = """
    MATCH (re:REVIEWS)
    WHERE NOT EXISTS ((re)-[:POSTED]->(:USER))
    RETURN re
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there are breweries that are not linked to any beers 
query = """
    MATCH (br:BREWERY)
    WHERE NOT EXISTS ((br)-[:BREWED]->(:BEERS))
    RETURN br
"""
result = execute_read(driver, query)

pprint(result)

- There are breweries not associated w/ any city.
- There are cities not associated w/ any country.
- There are reviews not associated w/ any user or beer.
- There are beers not associated w/ any brewery.

## 1.3 EXERCISE SUMMARY
- Indexes: Style, user, countries and cities id is the property name.Reviews, Breweries, Beers id id the property id.
- Labels: COUNTRIES, CITIES, BREWERIES, BEERS, REVIEWS, STYLE and USER.

- Relationships:
    - Relationship names: REVIEWED, BREWED, IN, HAS_STYLE and POSTED.
    - Relationships are all unidrectional, which mnakes sense.
    - Relationships have no properties. 
    - There are several isolated nodes. We will try to connect them, and delete the ones that we can't connect.

- Properties:
    - COUNTRIES, CITIES, USER and STYLE only property: ‘name’.
    - REVIEWS properties: 'text', 'smell', 'look', 'taste', 'feel', 'overall', 'beer_id', 'id', 'date', 'score'
    - There are several node w/ label REVIEWS that have no properties.
    - BREWERIES properties: 'notes', 'types', 'id', 'name', 'state'
    - BEERS properties: notes', 'abv', 'name', 'state', 'id', 'retired', 'availability', 'brewery_id'

- Inconsistencies found:
    - Duplicated nodes: There are several nodes with the label BEERS, COUNTRIES, CITIES and REVIEWS duplicated for all properties.
    - There are breweries not associated w/ any city.
    - There are cities not associated w/ any country.
    - There are reviews not associated w/ any user or beer.
    - There are beers not associated w/ any brewery.

# 2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema.

### Adjustments to be made

- **2.1.** Connect isolated nodes where possible and delete the ones where it is not possible
    - Try to connect BEERS with BREWERIES if possible based on BEERS properties
    - Try to connect CITIES with COUNTRIES if possible
    - Associate BREWERIES with CITIES if possible.
- **2.2.** Delete nodes that are duplicated for all properties
- **2.3.** Question: Should one beer only be associated to one brewery? Maybe many breweries can produce the same beer
- **2.4.** Change REVIEWS - [POSTED] -> USERS to REVIEWS - [POSTED_BY] -> USERS
- **2.5.** Connecting BEER to COUNTRY so that we know what beers are available in each country directly.

### **2.1.** Connecting isolated NODES where possible. Deleting the ones that can't be connected.

In [ ]:
# Checking isolated nodes for REVIEWS before we do anything

query = """
    MATCH (n)
    WHERE "REVIEWS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
# Connecting isolated REVIEWS nodes where possible
# Connecting BEERS - [REVIEWED] -> REVIEWS

query = """
MATCH (r:REVIEWS)
WITH r
MATCH (b:BEER WHERE b.id = r.beer_id)
CREATE (b)-[:REVIEWED]->(r)
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking results

query = """
    MATCH (n)
    WHERE "REVIEWS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

We were not able to connect any nodes. We thought that BEER.id could match with REVIEWED.beer_id but that doesn't seem to be the case.

In [ ]:
# Deleting nodes that we could not connect

query = """
    MATCH (n)
    WHERE "REVIEWS" IN labels(n) AND NOT EXISTS ((n)--())
    DELETE n
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking results

query = """
    MATCH (n)
    WHERE "REVIEWS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

We successfully deleted the isolated REVIEWS nodes.

In [ ]:
# Checking isolated nodes for USER before we do anything

query = """
    MATCH (n)
    WHERE "USER" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
"""

result = execute_read(driver, query)
pprint(result)

For users, the only node property is 'name', so we can't connect them. We will be deleting those.

In [ ]:
# Deleting nodes that we could not connect

query = """
    MATCH (n)
    WHERE "USER" IN labels(n) AND NOT EXISTS ((n)--())
    DELETE n
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking result

query = """
    MATCH (n)
    WHERE "USER" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
"""

result = execute_read(driver, query)
pprint(result)

Sucessfully deleted isolated USERS nodes.

In [ ]:
# Checking isolated nodes for STYLE before we do anything

query = """
    MATCH (n)
    WHERE "STYLE" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

There are no isolated STYLE nodes.

In [ ]:
# Checking isolated nodes for BEERS before we do anything

query = """
    MATCH (n)
    WHERE "BEERS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
# Count of isolated nodes for beers before changes
query = """
    MATCH (n)
    WHERE "BEERS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN count(n)
    """

result = execute_read(driver, query)
pprint(result)

BREWERIES - [r] -> BEERS nodes connection attempt

In [ ]:
# Number of BREWERIES - r -> BEERS connections existing before trying to connect the isolated nodes

query = """
MATCH (b:BREWERIES)-[r:BREWED]->(beer:BEERS)
RETURN COUNT(r) AS total_relationships;
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Trying to connect isolated BEERS nodes to BREWERIES: BREWERIES [BREWED] BEERS based on properties

query = """
MATCH (beer:BEERS)
MATCH (b:BREWERIES)
WHERE b.id = beer.brewery_id
MERGE (b)-[:BREWED]->(beer);
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Number of connections existing AFTER trying to connect the isolated nodes

query = """
MATCH (b:BREWERIES)-[r:BREWED]->(beer:BEERS)
RETURN COUNT(r) AS total_relationships;
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking isolated nodes for BEERS AFTER we do the process

query = """
    MATCH (n)
    WHERE "BEERS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN n
    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
# Count of isolated nodes for beers AFTER changes
query = """
    MATCH (n)
    WHERE "BEERS" IN labels(n) AND NOT EXISTS ((n)--())
    RETURN count(n)
    """

result = execute_read(driver, query)
pprint(result)

**We were able to connect all isolated BEER nodes to BREWERIES.**

We can't connect CITIES with COUNTRIES as their only property is name. The only thing left to do with the isolated nodes in these two is to delete them.

### After this point, it seems impossible to connect any other nodes taking into account the properties that we have. E.g. for BREWERIES we have the property state but not City. And City only has property name, not property State.<br>
### As such, we will simply delete all the remaining isolated nodes.

In [ ]:
# Deleting all nodes that we could not connect

query = """
MATCH (n)
WHERE NOT (n)--()
DELETE n;
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking if there are still isolated nodes

query = """
MATCH (n)
WHERE NOT (n)--()
RETURN count(n);
"""

result = execute_read(driver, query)
pprint(result)

## 2.2 Deleting all duplicate nodes

### Delete nodes duplicated for all properties

In [ ]:
# Collecting the nodes and identifying duplicates

query = """
MATCH (n)
WITH n, LABELS(n) AS labels, KEYS(n) AS keys, [k IN KEYS(n) | n[k]] AS values
WITH labels, values, COLLECT(n) AS nodes
WHERE SIZE(nodes) > 1
RETURN labels, values, nodes
"""

result = execute_read(driver, query)
pprint(result)

In [ ]:
# Removing duplicate nodes, keeping only one
query = """
MATCH (n)
WITH n, LABELS(n) AS labels, KEYS(n) AS keys, [k IN KEYS(n) | n[k]] AS values
WITH labels, values, COLLECT(n) AS nodes
WHERE SIZE(nodes) > 1
CALL {
  WITH nodes
  UNWIND nodes[1..] AS duplicate  // Keep the first node, delete the rest
  DETACH DELETE duplicate
}
RETURN COUNT(*) AS deleted_nodes
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking results

# Collecting the nodes and identifying duplicates

query = """
MATCH (n)
WITH n, LABELS(n) AS labels, KEYS(n) AS keys, [k IN KEYS(n) | n[k]] AS values
WITH labels, values, COLLECT(n) AS nodes
WHERE SIZE(nodes) > 1
RETURN labels, values, nodes
"""

result = execute_read(driver, query)
pprint(result)

### **2.3.** Do all BEERS have at least one BREWERY associated? Check ´brewery_id´ property

In [ ]:
# Checking if all BEERS have BREWERY associated
query = """
MATCH (b:BEERS)
WHERE NOT EXISTS(b.brewery_id) OR b.brewery_id IS NULL
RETURN b
"""

result = execute_read(driver, query)
pprint(result)

- Since all BEERS have a BREWERY associated, we do not need to make any changes here.

### **2.4** Changing [POSTED] to [POSTED_BY]

In [ ]:
# Initial [POSTED] relationships
query = """
MATCH (re:REVIEWS)-[r:POSTED]->(u:USER)
RETURN COUNT(r)
"""

result = execute_read(driver, query)
pprint(result)

In [ ]:
deleted_count = 1

query = """ 
MATCH (re:REVIEWS)-[r:POSTED]->(u:USER)
WITH re, u, r LIMIT 100000  
CREATE (re) -[:POSTED_BY {}] -> (u)
DELETE r
RETURN COUNT(r) AS deletedCount
"""

while deleted_count != 0:
    start_time = time.time()

    result = execute_write(driver, query)

    print(f"Result:{result}")
    
    end_time = time.time()
    
    run_time = end_time - start_time

    print(f"Time taken for this iteration: {round(run_time,2)} seconds")
    
    deleted_count = result[0][0]

In [ ]:
# Checking final result
query = """
MATCH (re:REVIEWS)-[r]->(u:USER)
RETURN DISTINCT type(r) AS relationship_type
"""

result = execute_read(driver, query)
pprint(result)

In [ ]:
query = """
MATCH (re:REVIEWS)-[r:POSTED_BY]->(u:USER)
RETURN COUNT(r)
"""

result = execute_read(driver, query)
pprint(result)

We have successfully replaced the [POSTED] relationship by [POSTED_BY].

### **2.5** Creating [BEERS] - [AVAILABLE_IN] - [COUNTRIES]

In [ ]:
# Creating new relationship

query = """
MATCH (b:BREWERIES)-[:IN]->(c:COUNTRIES), (b)-[:BREWED]->(beer:BEERS)
MERGE (beer)-[:AVAILABLE_IN]->(c)
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking all one-directional relationships after creating this one

query = """
MATCH (a)-[r]->(b)
WHERE NOT EXISTS {
    MATCH (b)-[r_reverse]->(a)
}
RETURN DISTINCT
  type(r) AS relationship_type,
  labels(a) AS start_node_labels,
  labels(b) AS end_node_labels
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Final schema visualization

query = """
    call db.schema.visualization;
    """

result = execute_read(driver, query)

pprint(result)


### **Summary of Changes to Dabatase**
- Created a link between BEERS and COUNTRIES 
- Deleted isolated nodes that could not be connected
- Deleted all duplicated nodes.

### 3.1 How many reviews has the beer with the most reviews?

In [ ]:
query = """
        MATCH (b:BEERS)-[r]->(:REVIEWS)
        WITH b, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1
        RETURN b.name, review_count


    """

result = execute_read(driver, query)
pprint(result)


OR

In [ ]:
query = """
        MATCH (b:BEERS)-[r]->(:REVIEWS)
        WITH b.name AS beer_name, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1 
        RETURN beer_name, review_count

    """

result = execute_read(driver, query)
pprint(result)

### 3.2 Which three users wrote the most reviews about beers?

In [ ]:
query = """
        MATCH (r:REVIEWS)-[p:POSTED]->(u:USER)
        WITH u, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 3
        RETURN u.name
    """

result = execute_read(driver, query)
pprint(result)

### *REVIEW* 3.3 Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'. *REVIEW*

In [ ]:
words = ['fruit', 'complex', 'nutty', 'dark']

query = f"""
        WITH {words} AS words

        MATCH (u:BEERS)-[r:REVIEWED]->(rs:REVIEWS)
        

        WHERE any(word IN words WHERE rs.text CONTAINS word)

        RETURN DISTINCT u.name
    """

result = execute_read(driver, query)

pprint(result)

### 3.5 Which top three breweries produce the largest variety of beer styles?

In [ ]:
query = """
        MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[h:HAS_STYLE]->(s:STYLE)
        RETURN br.name, count(s)
        ORDER BY count(s) DESC
        LIMIT 3 
    """

result = execute_read(driver, query)
pprint(result)

### 3.5 Which country produces the most beer styles?

In [ ]:
query = """
        MATCH (c:COUNTRIES)<-[:IN]-(ci:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        RETURN c.name, count(s)
        ORDER BY count(s) DESC
        LIMIT 1
    """

result = execute_read(driver, query)
pprint(result)

### **4.1.** Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.

In [ ]:
#without indexing
query = """ 
    PROFILE
    MATCH (be:BEERS)-[:BREWED]-(br:BREWERIES)
    WHERE be.abv <> 'nan'
    RETURN be.name AS Beer, be.abv AS ABV, br.name AS Brewery
    ORDER BY be.abv DESC
    LIMIT 5
"""

result = execute_read(driver, query)
pprint(result)

In [ ]:
query = """
        CREATE INDEX FOR (b:BEERS) ON (b.abv)
    """

result = execute_write(driver, query)
pprint(result)

In [ ]:
#with indexing
query = """
        PROFILE
        MATCH (be:BEERS)-[:BREWED]-(br:BREWERIES)
        WHERE be.abv <> 'nan'
        RETURN be.name AS Beer, be.abv AS ABV, br.name AS Brewery
        ORDER BY be.abv DESC
        LIMIT 5
    """

result = execute_read(driver, query)
pprint(result)

### **4.2.** Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.

In [ ]:
query = """
    // 3rd most productive user
    MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)
    WITH u, COUNT(r) AS review_count
    ORDER BY review_count DESC
    SKIP 2 LIMIT 1 // Skip the top 2 and get the 3rd most productive user
    WITH u

    // Average smell + feel score for each BEERS STYLE, filtered by the 3rd most productive user
    MATCH (u)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WHERE toFloat(r.smell) IS NOT NULL AND toFloat(r.feel) IS NOT NULL
    WITH s.name AS style, ROUND(AVG(toFloat(r.smell) + toFloat(r.feel)), 2) AS avg_smell_feel
    ORDER BY avg_smell_feel DESC

    // Top 5 styles
    LIMIT 5
    RETURN style, avg_smell_feel
"""

result = execute_read(driver, query)

pprint(result)

### **5.1** Which two countries are most similiar when it comes to their top five most produced Beer styles?

In [ ]:
query = """
        CALL gds.graph.project.cypher(
            'countryBeerGraph',
            "MATCH (c:COUNTRIES) RETURN id(c) AS id",
            "MATCH (b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(style:STYLE)
            WITH beer, style, COUNT(*) AS styleCount
            MATCH (c1:COUNTRIES)<-[:IN]-(b)-[:IN]->(c2:COUNTRIES)
            RETURN id(c1) AS source, id(c2) AS target, style, styleCount"
)
    """

result = execute_read(driver, query)
pprint(result)


In [ ]:
#First getting the top 5 most produced Beer styles for each country

query = """
        MATCH (c:COUNTRIES)<-[:IN]-(ci:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WITH c.name AS country, s.name AS beer_style, COUNT(*) AS count
        ORDER BY country, count DESC
        WITH country, COLLECT(beer_style)[..5] AS top_styles
        RETURN country, top_styles;
    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
try:
    query = """
            CALL gds.graph.drop('countryBeerGraph') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

In [ ]:
query = """
        CALL gds.graph.project.cypher(
            'countryBeerGraph',
            "MATCH (c:COUNTRIES) RETURN id(c) AS id",
            "
            MATCH (b1:BREWERIES)-[:BREWED]->(beer1:BEERS)-[:HAS_STYLE]->(s:STYLE)<-[:HAS_STYLE]-(beer2:BEERS)<-[:BREWED]-(b2:BREWERIES)
            WITH b1, b2, COUNT(DISTINCT s) AS weight
            WHERE weight > 0
            MATCH (c1:COUNTRIES)<-[:IN]-(b1)-[:IN]->(c2:COUNTRIES)
            WHERE c1 <> c2
            RETURN id(c1) AS source, id(c2) AS target, weight"
        )

    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
#Apply the Node Similarity Algorithm 
#Countries are nodes and edges represent similarities based on shared beer styles
#edges have weights based on the number of shared beer styles

#1-Create the graph for similarity analysis
query = """
        CALL gds.graph.project.cypher(
            'countryBeerGraph',
            "MATCH (c:COUNTRIES) RETURN id(c) AS id",
            "MATCH (c1:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(b1:BREWERIES)-[:BREWED]->(beer1:BEERS)-[:HAS_STYLE]->(s:STYLE),
                (c2:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(b2:BREWERIES)-[:BREWED]->(beer2:BEERS)-[:HAS_STYLE]->(s:STYLE)
            WHERE c1 <> c2
            WITH c1, c2, COUNT(DISTINCT s) AS weight 
            WHERE weight > 0 
            RETURN id(c1) AS source, id(c2) AS target, weight" 
        );

    """

result = execute_read(driver, query)
pprint(result)

In [ ]:
query = """
        CALL gds.graph.list() YIELD graphName, nodeCount, relationshipCount
        WHERE graphName = 'countryBeerGraph';



    """

result = execute_read(driver, query)
pprint(result)


In [ ]:
#Apply the Node Similarity Algorithm 

#2- Run the algorithm

query = """
        CALL gds.nodeSimilarity.stream('countryBeerGraph', { similarityMetric: 'JACCARD' })
        YIELD node1, node2, similarity
        RETURN gds.util.asNode(node1).name AS country1,
            gds.util.asNode(node2).name AS country2,
            similarity
        ORDER BY similarity DESC
        LIMIT 1

    """

result = execute_read(driver, query)
pprint(result)

### **5.2.** Which beer is the most popular when considering the number of users who reviewed it? 

In [ ]:
try:
    query = """
            CALL gds.graph.drop('beer_popularity') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

In [ ]:
# Create a graph projection for beers and reviews

query = """

CALL gds.graph.project(
    'beer_popularity',
    ['BEERS', 'REVIEWS'],
    {
        REVIEWED: {
            orientation: 'UNDIRECTED'
        }
    }
)
"""
execute_write(driver, query)

In [ ]:
# Compute Degree Centrality for beers

query = """

CALL gds.degree.stream('beer_popularity')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS beer_name, score AS num_reviews
ORDER BY num_reviews DESC
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result)

### **5.4.** Which user is the most influential when it comes to reviews of distinct beers by style?

In [ ]:
try:
    query = """
            CALL gds.graph.drop('user_influence_graph') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

In [ ]:
try:
    query = """
            CALL gds.graph.project.cypher(
            'user_influence_graph',
            'MATCH (u:USER) RETURN id(u) AS id
            UNION
            MATCH (s:STYLE) RETURN id(s) AS id',
            'MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
            RETURN id(u) AS source, id(s) AS target'
            );
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

In [ ]:
try:
    query = """
        CALL gds.pageRank.stream('user_influence_graph')
        YIELD nodeId, score
        WHERE 'USER' IN labels(gds.util.asNode(nodeId))
        RETURN gds.util.asNode(nodeId).name AS user, score AS influence_score
        ORDER BY influence_score DESC
        LIMIT 20;
    """
    result = execute_read(driver, query)
    pprint(result)

except Exception as e:
    pprint(e)